In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

In [ ]:
df = pd.read_csv("../rawdata/hololivecomments.csv")
del df['subreddit.1']

In [ ]:
df = df.dropna(axis=0, how="any", subset=None, inplace=False)

In [ ]:
# Sort data berdasarkan waktu posting dan mengubah epoch value menjadi datetime UTC+7

df.sort_values(by = ["created_utc"], inplace = True, ascending = True)
df['created_utc'] = pd.DatetimeIndex(pd.to_datetime(df['created_utc'], unit='s')).tz_localize('UTC').tz_convert('Asia/Jakarta')

In [ ]:
display(df.head(5))
display(df.tail(5))
display(df.info())

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

STOP_WORDS = stopwords.words()

# removing the emojies
# https://www.kaggle.com/alankritamishra/covid-19-tweet-sentiment-analysis#Sentiment-analysis
EMOJI_PATTERN = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

tokenSpace = tokenize.WhitespaceTokenizer()

def cleaning(text):
    """
    Convert to lowercase.
    Rremove URL links, special characters and punctuation.
    Tokenize and remove stop words.
    """
    # text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('[’“”…]', '', text)

    text = EMOJI_PATTERN.sub(r'', text)

    # removing the stop-words
    text_tokens = word_tokenize(text)
    tokens_without_sw = [
        word for word in text_tokens if not word in STOP_WORDS]
    filtered_sentence = (" ").join(tokens_without_sw)
    text = filtered_sentence

    return text


def counter(text, columnText, quantity): #counter(df[df['score'] == 1], 'body', 10)
    allWords = ' '.join([text for text in text[columnText].astype('str')])
    tokenPhrase = tokenSpace.tokenize(allWords)
    frequency = nltk.FreqDist(tokenPhrase) 
    dfFrequency = pd.DataFrame({"Word": list(frequency.keys()), "Frequency": list(frequency.values())}) 
    
    dfFrequency = dfFrequency.nlargest(columns = "Frequency", n = quantity)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = dfFrequency, x = "Word", y = "Frequency", palette="deep")
    ax.set(ylabel = "Count")
    plt.xticks(rotation='horizontal')
    plt.show()

In [ ]:
df2 = df.copy()

df2['body'] = df2['body'].apply(cleaning)
df2.to_csv("../rawdata/hololivecomments_cleaned.csv", index=False)